# Memory footprint estimation

Below are calculations on how much GPU memory is approximately required to run the [DAVEnet model](https://github.com/dharwath/DAVEnet-pytorch) (Harwath et al. 2018).

In [1]:
# For importing the models folder
import sys
sys.path.append('/m/home/home4/44/virkkua1/unix/PlacesAudio_project/DAVEnet')

import torch
import torchvision
from torchsummary import summary

import models

The model consists of two CNNs, an audio and an image branch. The audio branch is a 5-layer CNN that generates audio embeddings from spoken captions. The image branch is a standard VGG16 where the final maxpool is replaced with 2D convolutional layer.

In [2]:
audio_model = models.Davenet()
audio_model.cuda()

Davenet(
  (batchnorm1): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): Conv2d(1, 128, kernel_size=(40, 1), stride=(1, 1))
  (conv2): Conv2d(128, 256, kernel_size=(1, 11), stride=(1, 1), padding=(0, 5))
  (conv3): Conv2d(256, 512, kernel_size=(1, 17), stride=(1, 1), padding=(0, 8))
  (conv4): Conv2d(512, 512, kernel_size=(1, 17), stride=(1, 1), padding=(0, 8))
  (conv5): Conv2d(512, 1024, kernel_size=(1, 17), stride=(1, 1), padding=(0, 8))
  (pool): MaxPool2d(kernel_size=(1, 3), stride=(1, 2), padding=(0, 1), dilation=1, ceil_mode=False)
)

In [3]:
image_model = models.VGG16(pretrained=True)
image_model.cuda()

VGG16(
  (image_model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilat

Parameter counts are computed by the torchsummary module, which produces Keras-like network summaries.

In [4]:
# The second parameter is input size.
summary(image_model, (3, 224, 224))
summary(audio_model, (40, 1024))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

To estimate the total memory footprint of a model, we need to add together the input size, forward *and* backward pass sizes (AKA feature + gradient maps or channels) and parameters' size (AKA weights and bias). We can see above that torchsummary can do this for us. But to understand where the numbers come from, let's do a sanity check below and compute the estimates for the audio network ourselves.

The input size is computed as a product of the input dimensions. For forward pass size, dimensions of each layer are multiplied and the resulting products are summed together. Backward pass size is the same as forward pass, so forward pass can be multiplied by two. The weights/parameters for each layer are a product of the kernel size and input and output depths (i.e. the number of feature maps/channels). The potential bias vector size is added on top of the product. Parameters are naturally computed only for layers which have them, in this case convolutional and batch norm layers. In the end, all parameters are summed together. Finally, to convert each result to megabytes, they need to be multiplied by 4 bytes (values are presumably stored as float32, so each value uses 32 bits i.e. 4 bytes) and then divided by the square of 1024 (torchsummary uses binary notation). 

$$\text{Input:} \ \ \frac{40*1024*4\text{B}}{1024^2} = 0.15625 \text{MB} \approx 0.16 \text{MB}$$

$$\text{Forward+backward pass:} \ \ \frac{40*1024+128*1024+256*1024+256*512+512^2+2*512*256+512*128+1024*128+1024*64}{1024^2}*4\text{B}*2 = 10.3125 \text{MB} \approx 10.31 \text{MB}$$

$$\text{Parameters:} \ \ \frac{2+1*128*40+128+128*256*11+256+256*512*17+512+512^2*17+512+512*1024*17+1024}{1024^2}*4\text{B} \approx 60.90 \text{MB}$$

However, these estimates do not take batch size into account. When computing on a GPU, one batch is loaded to memory at a time. So when the batch size is for example 10, input size needs to be multiplied by 10. The same applies to forward and backward pass sizes, because activation and gradient maps are different for each input item in a batch. Parameters stay the same across different inputs, so the parameter size can be left untouched.

In the DAVEnet paper, authors write that they train the model with batch size 128. Supplying the `batch_size` argument to the `summary()` gives us the estimate of

$$28273.64 \ \text{MB} = \frac{28273.64 \ \text{MB}}{1024} \approx 27.61 \ \text{GB} $$

for the image model and

$$1400.90 \ \text{MB} = \frac{1400.90 \ \text{MB}}{1024} \approx 1.37 \ \text{GB}$$

for the audio model. 

In [5]:
summary(image_model, (3, 224, 224), batch_size=128)
summary(audio_model, (40, 1024), batch_size=128)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1        [128, 64, 224, 224]           1,792
              ReLU-2        [128, 64, 224, 224]               0
            Conv2d-3        [128, 64, 224, 224]          36,928
              ReLU-4        [128, 64, 224, 224]               0
         MaxPool2d-5        [128, 64, 112, 112]               0
            Conv2d-6       [128, 128, 112, 112]          73,856
              ReLU-7       [128, 128, 112, 112]               0
            Conv2d-8       [128, 128, 112, 112]         147,584
              ReLU-9       [128, 128, 112, 112]               0
        MaxPool2d-10         [128, 128, 56, 56]               0
           Conv2d-11         [128, 256, 56, 56]         295,168
             ReLU-12         [128, 256, 56, 56]               0
           Conv2d-13         [128, 256, 56, 56]         590,080
             ReLU-14         [128, 256,

### Cosine annealing test



In [24]:
trainables = [p for p in audio_model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(trainables, 0.1, momentum=0.9, weight_decay=5e-7)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 100, eta_min=1e-6)

In [25]:
for i in range(800):
    print(i, ": ", scheduler.get_lr())
    scheduler.step()

0 :  [0.1]
1 :  [0.09997532826500641]
2 :  [0.09990133740804936]
3 :  [0.09977810044917171]
4 :  [0.09960573900837325]
5 :  [0.0993844231855866]
6 :  [0.09911437139280908]
7 :  [0.09879585013855641]
8 :  [0.098429173764851]
9 :  [0.09801470413700432]
10 :  [0.09755285028649954]
11 :  [0.0970440680073268]
12 :  [0.09648885940616962]
13 :  [0.09588777240688622]
14 :  [0.09524140020977476]
15 :  [0.0945503807061563]
16 :  [0.09381539584885315]
17 :  [0.09303717097918368]
18 :  [0.09221647411113801]
19 :  [0.09135411517344096]
20 :  [0.0904509452102502]
21 :  [0.08950785554127832]
22 :  [0.08852577688216808]
23 :  [0.08750567842598816]
24 :  [0.08644856688675687]
25 :  [0.08535548550593679]
26 :  [0.08422751302288148]
27 :  [0.08306576261024994]
28 :  [0.08187138077543962]
29 :  [0.08064554622912201]
30 :  [0.07938946872199752]
31 :  [0.07810438785091763]
32 :  [0.07679157183555234]
33 :  [0.07545231626681069]
34 :  [0.07408794282824871]
35 :  [0.07269979799172747]
36 :  [0.071289251688607